In [1]:
import numpy as np
import random
import math

In [2]:
LIMITS = 100
DIMENSION = 30
MAX_FITNESS = 1
MAX_ITER = 1000
SAMPLE_SIZE = 1
INITIAL_SAMPLE_SIZE = 50

In [3]:
# Diccionario para no recalcular fitness:
dic = {}

In [4]:
def esfera(sol):
    sum = 0
    for xi in sol:
        sum += xi ** 2
    return sum

In [5]:
def fitness(sol):
    maxi = DIMENSION * LIMITS * LIMITS
    assert(maxi > 0)
    esf = esfera(sol)
    assert(esf <= maxi)
    return (maxi - esf) / maxi

In [6]:
def f(sol):
    fit = dic.get(tuple(sol))
    if fit == None:
        fit = fitness(sol)
        dic[tuple(sol)] = fit
    print(f"FITNESS de {sol} = {fit}")
    return fit

In [7]:
def manhattan_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return sum([abs(sol1[i] - sol2[i]) for i in range(n)])

In [8]:
def euclidean_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return math.sqrt(sum([(sol1[i] - sol2[i]) ** 2 for i in range(n)]))

In [9]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False)

In [10]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    assert(d > 0) # Creo que todo está preparado para no samplear dos iguales.
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [11]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.25

In [12]:
def search_2_nearest(sol, A, dist):
    card = len(A)
    assert(card > 1)
    first_distance = dist(sol, A[0])
    second_distance = dist(sol, A[1])
    if first_distance > second_distance:
        nearest = A[1]
        second = A[0]
        min_distance = second_distance
        sec_min_distance = first_distance
    else:
        nearest = A[0]
        second = A[1]
        min_distance = first_distance
        sec_min_distance = second_distance
    for elem in A[2:]:
        distance = dist(sol, elem)
        if distance < sec_min_distance:
            if distance < min_distance:
                sec_min_distance = min_distance
                min_distance = distance
                second = nearest
                nearest = elem
            else:
                sec_min_distance = distance
                second = elem
    return [nearest, second]

In [13]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for sol in A:
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [14]:
def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol, point, A, dist)
        fit = f(point)
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    print(f"\n\n g: {sum}\n\n")
    return sum

In [15]:
def sample(k, A, dist, skip = []):
    n = DIMENSION
    sample = []
    while len(sample) < k:
        x = [random.randint(0, LIMITS) for _ in range(n)]
        if x in sample or x in skip:
            continue
        #y = random.randint(0, MAX_FITNESS)
        #y = random.uniform(0, MAX_FITNESS)
        # Para fitness al cuadrado:
        y = random.uniform(0, MAX_FITNESS ** 3)
        set_points = search_2_nearest(x, A, dist)
        g_val = g(x, set_points, dist)
        # Para fitness al cuadrado:
        g_val = g_val ** 3
        if g_val > y:
            #print(fitness(x))
            print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            sample.append(x)
    return sample

In [16]:
def init(k):
    n = DIMENSION
    init = []
    while len(init) < k:
        sol = [random.randint(0, LIMITS) for _ in range(n)]
        if sol not in init: # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [17]:
def max_f(A):
    assert(len(A) > 0)
    max_value = f(A[0])
    max_elem = A[0]
    for elem in A[1:]:
        fit = f(elem)
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [18]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [21]:
elem, fit = alg(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, initial_sample_size = INITIAL_SAMPLE_SIZE, dist = euclidean_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem}, con volumen {esfera(elem)}.')

iter 0
FITNESS de [84, 38, 81, 1, 34, 64, 23, 94, 45, 22, 86, 10, 3, 37, 26, 56, 34, 46, 76, 65, 84, 94, 31, 84, 63, 24, 84, 28, 87, 37] = 0.6554033333333333
FITNESS de [86, 46, 89, 15, 18, 38, 83, 47, 65, 42, 67, 60, 35, 67, 89, 97, 55, 56, 94, 33, 3, 97, 26, 10, 52, 56, 63, 63, 75, 84] = 0.6053833333333334


 g: 0.6310874783043908


Muestra añadida [44, 13, 98, 54, 34, 22, 16, 37, 73, 68, 82, 56, 13, 62, 58, 93, 2, 37, 79, 85, 91, 97, 4, 54, 94, 68, 62, 81, 94, 56]
con valor de g: 0.25134409683422804 y valor de y: 0.0682158847940999
iter 1
FITNESS de [88, 14, 38, 96, 38, 72, 35, 11, 73, 21, 7, 9, 16, 10, 17, 49, 28, 70, 59, 77, 66, 2, 44, 42, 52, 27, 26, 32, 17, 60] = 0.7743466666666666
FITNESS de [53, 3, 41, 81, 32, 24, 55, 38, 34, 35, 13, 28, 25, 100, 32, 57, 91, 69, 76, 10, 84, 12, 81, 60, 66, 3, 14, 26, 32, 12] = 0.73985


 g: 0.757145654880055


Muestra añadida [87, 9, 31, 83, 37, 31, 62, 32, 66, 50, 45, 48, 0, 59, 76, 26, 61, 34, 100, 25, 77, 10, 10, 27, 27, 60, 23, 79, 9, 32]
